In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from construct_network import Construct_Network

## 1. Prepare weighted PPI

In [2]:
from prepared_weighted_ppi import pre_weighted_ppi
ppi_pd = pd.read_csv('resource_data/PPI.csv')
suhuang_expressionp_pd = pd.read_csv('resource_data/SHVSM_expr_result_all.csv')
weighted_ppi_sif = pre_weighted_ppi(ppi_pd, suhuang_expressionp_pd)
# save weighted ppi out
weighted_ppi_sif.to_csv('resource_data/ppi_weighted.sif', header=None, sep='\t', index = None)

## 2. load weighted network to construct network

In [3]:
import pickle

g_obj = Construct_Network('resource_data/ppi_weighted.sif')
nodes_ppi = g_obj.G.nodes

# save out as dictionar to save time
with(open('resource_data/suhuang_weighted_g_obj.obj', 'wb')) as handle:
        pickle.dump(g_obj, handle)
        

In [4]:
import pickle
g_obj  = pickle.load(open('resource_data/suhuang_weighted_g_obj.obj', 'rb'))
nodes_ppi = g_obj.G.nodes

## 3. prepare pathways annotation

In [5]:
# from utitilize import prepare_pathway_genes_anno

# # prepare pathways matrix for heatmap annotation
# pathway_selected_path = 'source_data/weighted_rat_ppi/selected_pathways.xlsx'
# gene_expression_path = 'resource_data/SHVSM_diff_expr_gene_res.csv'
# gene_info, pathways_gene_matrix = prepare_pathway_genes_anno(nodes_ppi, pathway_selected_path, gene_expression_path)
# gene_info.to_csv('result/suhuang/target_gene_heat/gene_pathways_info.csv', index=None)
# pathways_gene_matrix.to_csv('result/suhuang/target_gene_heat/pathways_gene_matrix.csv')

## 4. prepare targets, CVA-genes, DEGs

In [6]:
from utitilize import prepare_target_anno, prepare_all_DEGs, prepare_cva_disease_gene_

# prepare annotation for target of ingredients
target_threshold = 950
ingre_anno_path = 'source_data/SUHUANG_h_ingre_.csv'
save_path = 'result/suhuang/target_gene_heat'
target_nodes = prepare_target_anno(950, ingre_anno_path, nodes_ppi, save_path)
# get all DEGs
DEG_path = 'source_data/weighted_rat_ppi/SHVSM_diff_expr_gene_res.csv'
DEG_nodes = prepare_all_DEGs(DEG_path, nodes_ppi)
# prepare CVA gene
disease_gene_path = 'source_data/weighted_rat_ppi/gold_CVA_related_gene.csv'
disease_nodes = prepare_cva_disease_gene_(disease_gene_path, nodes_ppi)

## 5. calculate LR score for disease genes

In [7]:
from cal_LR import cal_LR_disase_gene_DEG

#  calculate local radility score
DEG_dis_gene_distance_matrix, DEG_dis_gene_lr_score = cal_LR_disase_gene_DEG(g_obj.g, DEG_nodes, disease_nodes, weight=True)

# save out
DEG_dis_gene_distance_matrix.to_csv('result/suhuang/target_gene_dis/distance_disease_deg_pd_2.csv')
DEG_dis_gene_lr_score.to_csv('result/suhuang/target_gene_dis/group_diseasegene_LR.csv')

## 6. calculate heat diffusion score for target

In [10]:
from cal_heat_diffusion import cal_target_DEG_heat_score
taregt_DEG_path  = 'result/suhuang/target_gene_dis/target_gene_dis_all.csv'
EG_tar_dist_mat_binary = 'result/suhuang/target_gene_dis/DEG_taret_distance_matrix_binary.csv'

diffusion_result_pd_suhuang = 'result/suhuang/target_gene_dis/diffusion_result_pd_suhunag.csv'

suhuang_expressionp_pd = pd.read_csv('resource_data/SHVSM_expr_result_all.csv', index_col=0)

# Set the diffusion time parameter t
t = 0.1

DEG_tar_distance_matrix, DEG_tar_dist_mat_binary, diffusion_result_pd = cal_target_DEG_heat_score(g_obj.G,
                                                                                                  suhuang_expressionp_pd, 
                                                                                                  DEG_nodes,
                                                                                                  target_nodes,
                                                                                                  t,
                                                                                                  taregt_DEG_path)

DEG_tar_distance_matrix.to_csv(taregt_DEG_path)

EG_tar_dist_mat_binary.to_csv(EG_tar_dist_mat_binary)

diffusion_result_pd

cut threshold is 1.669029479
The loss is  61.592683223447
cuurent d [1.91472482 1.87915769 1.65853926 ... 1.87943502 1.87607394 1.868106  ]
The loss is  6.491076920688713
Converged after 2 iterations.
The final diffusion vector is: [1.91472482 1.87915769 1.65853926 ... 1.87943502 1.87607394 1.868106  ]


In [ ]:

# distance_disease_deg_pd = closest_distance_matrix(g_obj.g, deg_nodes_2, disease_nodes_2, weight=True)
# distance_disease_deg_pd.to_csv('result/suhuang/target_gene_dis/distance_disease_deg_pd_2.csv')